# Introduction

In this guide, we will walk you through building a powerful semantic search engine using [Couchbase](https://www.couchbase.com) as the backend database and [CrewAI](https://github.com/crewAIInc/crewAI) for agent-based RAG operations. CrewAI allows us to create specialized agents that can work together to handle different aspects of the RAG workflow, from document retrieval to response generation. This tutorial is designed to be beginner-friendly, with clear, step-by-step instructions that will equip you with the knowledge to create a fully functional semantic search system from scratch.

How to run this tutorial
----------------------
This tutorial is available as a Jupyter Notebook (.ipynb file) that you can run 
interactively. You can access the original notebook here.

You can either:
- Download the notebook file and run it on [Google Colab](https://colab.research.google.com)
- Run it on your system by setting up the Python environment

Before you start
---------------

1. Create and Deploy Your Free Tier Operational cluster on [Capella](https://cloud.couchbase.com/sign-up)
   - To get started with [Couchbase Capella](https://cloud.couchbase.com), create an account and use it to deploy 
     a forever free tier operational cluster
   - This account provides you with an environment where you can explore and learn 
     about Capella with no time constraint
   - To know more, please follow the [Getting Started Guide](https://docs.couchbase.com/cloud/get-started/create-account.html)

2. Couchbase Capella Configuration
   When running Couchbase using Capella, the following prerequisites need to be met:
   - Create the database credentials to access the [travel-sample bucket](https://docs.couchbase.com/server/current/manage/manage-settings/install-sample-buckets.html) (Read and Write) 
     used in the application
   - Allow access to the Cluster from the IP on which the application is running by following the [Network Security documentation](https://docs.couchbase.com/cloud/security/security.html#public-access)

# Setting the Stage: Installing Necessary Libraries

We'll install the following key libraries:
- `datasets`: For loading and managing our training data
- `langchain-couchbase`: To integrate Couchbase with LangChain for vector storage and caching
- `langchain-openai`: For accessing OpenAI's embedding and chat models
- `crewai`: To create and orchestrate our AI agents for RAG operations
- `python-dotenv`: For securely managing environment variables and API keys
- `tqdm`: For displaying progress bars during data processing

These libraries provide the foundation for building a semantic search engine with vector embeddings, 
database integration, and agent-based RAG capabilities.

In [1]:
%pip install --quiet datasets langchain-couchbase langchain-openai crewai python-dotenv tqdm

Note: you may need to restart the kernel to use updated packages.


# Importing Necessary Libraries
The script starts by importing a series of libraries required for various tasks, including handling JSON, logging, time tracking, Couchbase connections, embedding generation, and dataset loading.

In [2]:
import json
import logging
import os
import time
from datetime import timedelta

from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.diagnostics import PingState, ServiceType
from couchbase.exceptions import (CouchbaseException,
                                  InternalServerFailureException)
from couchbase.management.search import SearchIndex
from couchbase.options import ClusterOptions
from datasets import load_dataset
from dotenv import load_dotenv
from langchain.tools import Tool
from langchain_couchbase.vectorstores import CouchbaseVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from crewai import Agent, Crew, Process, Task

# Setup Logging
Logging is configured to track the progress of the script and capture any errors or warnings.

In [3]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Loading Sensitive Informnation
In this section, we prompt the user to input essential configuration settings needed. These settings include sensitive information like database credentials, and specific configuration names. Instead of hardcoding these details into the script, we request the user to provide them at runtime, ensuring flexibility and security.

The script uses environment variables to store sensitive information, enhancing the overall security and maintainability of your code by avoiding hardcoded values.

In [4]:
# Load environment variables
load_dotenv()

# Configuration
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY is not set")

CB_HOST = os.getenv('CB_HOST', 'couchbase://localhost')
CB_USERNAME = os.getenv('CB_USERNAME', 'Administrator')
CB_PASSWORD = os.getenv('CB_PASSWORD', 'password')
CB_BUCKET_NAME = os.getenv('CB_BUCKET_NAME', 'vector-search-testing')
INDEX_NAME = os.getenv('INDEX_NAME', 'vector_search_crew')
SCOPE_NAME = os.getenv('SCOPE_NAME', 'shared')
COLLECTION_NAME = os.getenv('COLLECTION_NAME', 'crew')

print("Configuration loaded successfully")

Configuration loaded successfully


# Connecting to the Couchbase Cluster
Connecting to a Couchbase cluster is the foundation of our project. Couchbase will serve as our primary data store, handling all the storage and retrieval operations required for our semantic search engine. By establishing this connection, we enable our application to interact with the database, allowing us to perform operations such as storing embeddings, querying data, and managing collections. This connection is the gateway through which all data will flow, so ensuring it's set up correctly is paramount.

In [5]:
# Connect to Couchbase
try:
    auth = PasswordAuthenticator(CB_USERNAME, CB_PASSWORD)
    options = ClusterOptions(auth)
    cluster = Cluster(CB_HOST, options)
    cluster.wait_until_ready(timedelta(seconds=5))
    print("Successfully connected to Couchbase")
except Exception as e:
    print(f"Failed to connect to Couchbase: {str(e)}")
    raise

Successfully connected to Couchbase


# Verifying Search Service Availability
 In this section, we verify that the Couchbase Search (FTS) service is available and responding correctly. This is a crucial check because our vector search functionality depends on it. If any issues are detected with the Search service, the function will raise an exception, allowing us to catch and handle problems early before attempting vector operations.


In [6]:
def check_search_service(cluster):
    """Verify search service availability using ping"""
    try:
        # Get ping result
        ping_result = cluster.ping()
        search_available = False
        
        # Check if search service is responding
        for service_type, endpoints in ping_result.endpoints.items():
            if service_type == ServiceType.Search:
                for endpoint in endpoints:
                    if endpoint.state == PingState.OK:
                        search_available = True
                        print(f"Search service is responding at: {endpoint.remote}")
                        break
                break

        if not search_available:
            raise RuntimeError("Search/FTS service not found or not responding")
        
        print("Search service check passed successfully")
    except Exception as e:
        print(f"Health check failed: {str(e)}")
        raise
try:
    check_search_service(cluster)
except Exception as e:
    print(f"Failed to check search service: {str(e)}")
    raise

Search service is responding at: 18.209.43.8:18094
Search service check passed successfully


# Setting Up Collections in Couchbase
In Couchbase, data is organized in buckets, which can be further divided into scopes and collections. Think of a collection as a table in a traditional SQL database. Before we can store any data, we need to ensure that our collections exist. If they don't, we must create them. This step is important because it prepares the database to handle the specific types of data our application will process. By setting up collections, we define the structure of our data storage, which is essential for efficient data retrieval and management.

Moreover, setting up collections allows us to isolate different types of data within the same bucket, providing a more organized and scalable data structure. This is particularly useful when dealing with large datasets, as it ensures that related data is stored together, making it easier to manage and query.

In [7]:

# Setup collections
try:
    bucket = cluster.bucket(CB_BUCKET_NAME)
    bucket_manager = bucket.collections()

    # Setup main collection
    collections = bucket_manager.get_all_scopes()
    collection_exists = any(
        scope.name == SCOPE_NAME and COLLECTION_NAME in [col.name for col in scope.collections]
        for scope in collections
    )

    if not collection_exists:
        try:
            bucket_manager.create_collection(SCOPE_NAME, COLLECTION_NAME)
            print(f"Collection '{COLLECTION_NAME}' created")
        except Exception as e:
            print(f"Failed to create collection '{COLLECTION_NAME}': {str(e)}")
            raise
    else:
        print(f"Collection '{COLLECTION_NAME}' already exists")

    # Create primary index
    try:
        cluster.query(
            f"CREATE PRIMARY INDEX IF NOT EXISTS ON `{CB_BUCKET_NAME}`.`{SCOPE_NAME}`.`{COLLECTION_NAME}`"
        ).execute()
        print(f"Primary index created for '{COLLECTION_NAME}'")
    except InternalServerFailureException as e:
        print(f"Failed to create primary index for '{COLLECTION_NAME}': {str(e)}")
        raise

    # Clear collection
    try:
        cluster.query(
            f"DELETE FROM `{CB_BUCKET_NAME}`.`{SCOPE_NAME}`.`{COLLECTION_NAME}`"
        ).execute()
        print(f"Collection '{COLLECTION_NAME}' cleared")
    except Exception as e:
        print(f"Failed to clear collection '{COLLECTION_NAME}': {str(e)}")
        raise

except Exception as e:
    print(f"An error occurred during setup: {str(e)}")
    raise

Collection 'crew' already exists
Primary index created for 'crew'
Collection 'crew' cleared


# Configuring and Initializing Couchbase Vector Search Index for Semantic Document Retrieval

Semantic search requires an efficient way to retrieve relevant documents based on a user's query. This is where the Couchbase Vector Search Index comes into play. In this step, we load the Vector Search Index definition from a JSON file, which specifies how the index should be structured. This includes the fields to be indexed, the dimensions of the vectors, and other parameters that determine how the search engine processes queries based on vector similarity.

This CrewAI vector search index configuration requires specific default settings to function properly. This tutorial uses the bucket named `vector-search-testing` with the scope `shared` and collection `crew`. The configuration is set up for vectors with exactly `1536 dimensions`, using `dot product` similarity and optimized for `recall`. If you want to use a different bucket, scope, or collection, you will need to modify the index configuration accordingly.

For more information on creating a vector search index, please follow the instructions at [Couchbase Vector Search Documentation](https://docs.couchbase.com/cloud/vector-search/create-vector-search-index-ui.html).

In [8]:
try:
    # Load index definition
    try:
        with open('crew_index.json', 'r') as file:
            index_definition = json.load(file)
    except FileNotFoundError as e:
        print(f"Error: crew_index.json file not found: {str(e)}")
        raise
    except json.JSONDecodeError as e:
        print(f"Error: Invalid JSON in crew_index.json: {str(e)}")
        raise
    except Exception as e:
        print(f"Error loading index definition: {str(e)}")
        raise

    # Setup vector search index
    try:
        scope_index_manager = cluster.bucket(CB_BUCKET_NAME).scope(SCOPE_NAME).search_indexes()

        # Check if index exists
        try:
            existing_indexes = scope_index_manager.get_all_indexes()
            index_exists = any(index.name == INDEX_NAME for index in existing_indexes)
        except InternalServerFailureException:
            # If no indexes exist yet, continue with creation
            index_exists = False
            print("No existing indexes found, proceeding with index creation")

        if index_exists:
            print(f"Index '{INDEX_NAME}' already exists")
        else:
            search_index = SearchIndex.from_json(index_definition)
            scope_index_manager.upsert_index(search_index)
            print(f"Index '{INDEX_NAME}' created")
    except InternalServerFailureException as e:
        print(f"Internal server error occurred while creating vector search index '{INDEX_NAME}'. This may indicate insufficient system resources or an issue with the Couchbase server configuration: {str(e)}")
        raise
    except CouchbaseException as e:
        print(f"A Couchbase-specific error occurred while managing vector search index '{INDEX_NAME}'. Please verify your cluster connectivity and index configuration settings: {str(e)}")
        raise
    except Exception as e:
        print(f"An unexpected error occurred while managing vector search index '{INDEX_NAME}'. This requires investigation of system logs for root cause analysis: {str(e)}")
        raise

except Exception as e:
    print(f"Fatal error in vector search index setup: {str(e)}")
    raise

Index 'vector_search_crew' already exists


# Setting Up OpenAI Components

This section initializes two key OpenAI components needed for our RAG system:

1. OpenAI Embeddings:
   - Uses the 'text-embedding-ada-002' model
   - Converts text into high-dimensional vector representations (embeddings)
   - These embeddings enable semantic search by capturing the meaning of text
   - Required for vector similarity search in Couchbase

2. ChatOpenAI Language Model:
   - Uses the 'gpt-4o' model
   - Temperature set to 0.0 for focused responses
   - Higher temperatures increase creativity and variation
   - Handles the actual text generation and responses
   - Acts as the brain of our RAG system for processing retrieved context

Both components require a valid OpenAI API key (OPENAI_API_KEY) for authentication.
The embeddings model is optimized for creating vector representations,
while the language model is optimized for understanding and generating human-like text.

In [9]:
# Initialize OpenAI components
embeddings = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY,
    model="text-embedding-ada-002"
)

llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model="gpt-4o",
    temperature=0.2
)

print("OpenAI components initialized")

OpenAI components initialized


# Setting Up the Couchbase Vector Store
A vector store is where we'll keep our embeddings. Unlike the FTS index, which is used for text-based search, the vector store is specifically designed to handle embeddings and perform similarity searches. When a user inputs a query, the search engine converts the query into an embedding and compares it against the embeddings stored in the vector store. This allows the engine to find documents that are semantically similar to the query, even if they don't contain the exact same words. By setting up the vector store in Couchbase, we create a powerful tool that enables our search engine to understand and retrieve information based on the meaning and context of the query, rather than just the specific words used.

In [10]:
# Setup vector store
vector_store = CouchbaseVectorStore(
    cluster=cluster,
    bucket_name=CB_BUCKET_NAME,
    scope_name=SCOPE_NAME,
    collection_name=COLLECTION_NAME,
    embedding=embeddings,
    index_name=INDEX_NAME,
)
print("Vector store initialized")

Vector store initialized


# Load the TREC Dataset
To build a search engine, we need data to search through. We use the TREC dataset, a well-known benchmark in the field of information retrieval. This dataset contains a wide variety of text data that we'll use to train our search engine. Loading the dataset is a crucial step because it provides the raw material that our search engine will work with. The quality and diversity of the data in the TREC dataset make it an excellent choice for testing and refining our search engine, ensuring that it can handle a wide range of queries effectively.

The TREC dataset's rich content allows us to simulate real-world scenarios where users ask complex questions, enabling us to fine-tune our search engine's ability to understand and respond to various types of queries.

In [11]:
# Load TREC dataset
trec = load_dataset('trec', split='train[:1000]')
print(f"Loaded {len(trec)} samples from TREC dataset")

try:
    batch_size = 50
    vector_store.add_texts(
        texts=trec['text'],
        batch_size=batch_size,
    )
    print(f"Added {len(trec)} documents to vector store")
except Exception as e:
    raise RuntimeError(f"Failed to save documents to vector store: {str(e)}")

Loaded 1000 samples from TREC dataset


2024-12-17 23:12:01 [INFO] HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Added 1000 documents to vector store


# Creating a Vector Search Tool
After loading our data into the vector store, we need to create a tool that can efficiently search through these vector embeddings. This involves two key components:

## Vector Retriever
The vector retriever is configured to perform similarity searches with specific parameters:
- k=8: Returns the 8 most similar documents
- fetch_k=20: Initially retrieves 20 candidates before filtering to the top 8
This two-stage approach helps balance between accuracy and performance.

## Search Tool
The search tool wraps the retriever in a user-friendly interface that:
- Accepts natural language queries
- Handles both string and structured query inputs
- Formats results with clear document separation
- Includes metadata for traceability

The tool is designed to integrate seamlessly with our AI agents, providing them with reliable access to our knowledge base through vector similarity search.


In [12]:
# Create vector retriever
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 8,
        "fetch_k": 20
    }
)

# Create search tool using retriever
search_tool = Tool(
    name="vector_search",
    func=lambda query: "\n\n".join([
        f"Document {i+1}:\n{'-'*40}\n{doc.page_content}"
        for i, doc in enumerate(retriever.invoke(
            query if isinstance(query, str) else str(query.get('query', ''))
        ))
    ]),
    description="""Search for relevant documents using vector similarity.
    Input should be a simple text query string.
    Returns a list of relevant document contents with metadata.
    Use this tool to find detailed information about topics."""
)

print("Vector search tool created")

Vector search tool created


# Creating CrewAI Agents

We'll create two specialized AI agents using the CrewAI framework to handle different aspects of our information retrieval and analysis system:

## Research Expert Agent
This agent is designed to:
- Execute semantic searches using our vector store
- Analyze and evaluate search results 
- Identify key information and insights
- Verify facts across multiple sources
- Synthesize findings into comprehensive research summaries

## Technical Writer Agent  
This agent is responsible for:
- Taking research findings and structuring them logically
- Converting technical concepts into clear explanations
- Ensuring proper citation and attribution
- Maintaining engaging yet informative tone
- Producing well-formatted final outputs

The agents work together in a coordinated way:
1. Research agent finds and analyzes relevant documents
2. Writer agent takes those findings and crafts polished responses
3. Both agents use a custom response template for consistent output

This multi-agent approach allows us to:
- Leverage specialized expertise for different tasks
- Maintain high quality through separation of concerns
- Create more comprehensive and reliable outputs
- Scale the system's capabilities efficiently

In [13]:
# Custom response template
response_template = """
Analysis Results
===============
{%- if .Response %}
{{ .Response }}
{%- endif %}

Sources
=======
{%- for tool in .Tools %}
* {{ tool.name }}
{%- endfor %}

Metadata
========
* Confidence: {{ .Confidence }}
* Analysis Time: {{ .ExecutionTime }}
"""

# Create research agent
researcher = Agent(
    role='Research Expert',
    goal='Find and analyze the most relevant documents to answer user queries accurately',
    backstory="""You are an expert researcher with deep knowledge in information retrieval 
    and analysis. Your expertise lies in finding, evaluating, and synthesizing information 
    from various sources. You have a keen eye for detail and can identify key insights 
    from complex documents. You always verify information across multiple sources and 
    provide comprehensive, accurate analyses.""",
    tools=[search_tool],
    llm=llm,
    verbose=True,
    memory=True,
    allow_delegation=False,
    response_template=response_template
)

# Create writer agent
writer = Agent(
    role='Technical Writer',
    goal='Generate clear, accurate, and well-structured responses based on research findings',
    backstory="""You are a skilled technical writer with expertise in making complex 
    information accessible and engaging. You excel at organizing information logically, 
    explaining technical concepts clearly, and creating well-structured documents. You 
    ensure all information is properly cited, accurate, and presented in a user-friendly 
    manner. You have a talent for maintaining the reader's interest while conveying 
    detailed technical information.""",
    llm=llm,
    verbose=True,
    memory=True,
    allow_delegation=False,
    response_template=response_template
)

print("Agents created successfully")

Agents created successfully


# Testing the Search System

Test the system with some example queries.

In [14]:
def process_query(query, researcher, writer):
    print(f"\nQuery: {query}")
    print("-" * 80)
    
    # Create tasks
    research_task = Task(
        description=f"Research and analyze information relevant to: {query}",
        agent=researcher,
        expected_output="A detailed analysis with key findings and supporting evidence"
    )
    
    writing_task = Task(
        description="Create a comprehensive and well-structured response",
        agent=writer,
        expected_output="A clear, comprehensive response that answers the query",
        context=[research_task]
    )
    
    # Create and execute crew
    crew = Crew(
        agents=[researcher, writer],
        tasks=[research_task, writing_task],
        process=Process.sequential,
        verbose=True,
        cache=True,
        planning=True
    )
    
    try:
        start_time = time.time()
        result = crew.kickoff()
        elapsed_time = time.time() - start_time
        
        print(f"\nQuery completed in {elapsed_time:.2f} seconds")
        print("=" * 80)
        print("RESPONSE")
        print("=" * 80)
        print(result)
        
        if hasattr(result, 'tasks_output'):
            print("\n" + "=" * 80)
            print("DETAILED TASK OUTPUTS")
            print("=" * 80)
            for task_output in result.tasks_output:
                print(f"\nTask: {task_output.description[:100]}...")
                print("-" * 40)
                print(f"Output: {task_output.raw}")
                print("-" * 40)
    except Exception as e:
        print(f"Error executing crew: {str(e)}")
        logging.error(f"Crew execution failed: {str(e)}", exc_info=True)

In [15]:
query = "What caused the 1929 Great Depression?"
process_query(query, researcher, writer)

23:12:07 - LiteLLM:INFO: utils.py:2765 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
2024-12-17 23:12:07 [INFO] 
LiteLLM completion() model= gpt-4o-mini; provider = openai



Query: What caused the 1929 Great Depression?
--------------------------------------------------------------------------------
 
[2024-12-17 23:12:07][INFO]: Planning the crew execution


2024-12-17 23:12:15 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
23:12:15 - LiteLLM:INFO: utils.py:894 - Wrapper: Completed Call, calling success_handler
2024-12-17 23:12:15 [INFO] Wrapper: Completed Call, calling success_handler
23:12:15 - LiteLLM:INFO: utils.py:2765 - 
LiteLLM completion() model= gpt-4o; provider = openai
2024-12-17 23:12:15 [INFO] 
LiteLLM completion() model= gpt-4o; provider = openai


# Agent: Research Expert
## Task: Research and analyze information relevant to: What caused the 1929 Great Depression?1. Initiate the task by clearly defining the research question: 'What caused the 1929 Great Depression?'
2. Use the vector_search tool to find relevant documents. Input the query string that accurately represents the research question.
3. Analyze the results from the vector_search tool. Review the list of documents returned with a focus on their content and metadata.
4. Collect key findings from the most relevant documents, noting any reliable sources, statistics, and historical facts that provide insight into the causes of the Great Depression.
5. Synthesize the findings into a cohesive analysis, ensuring it includes different perspectives and theories about the causes of the Great Depression, such as stock market crash, bank failures, reduced consumer spending, and international trade issues.
6. Prepare a summary of the analysis highlighting key findings and supportin

2024-12-17 23:12:17 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
23:12:17 - LiteLLM:INFO: utils.py:894 - Wrapper: Completed Call, calling success_handler
2024-12-17 23:12:17 [INFO] Wrapper: Completed Call, calling success_handler
2024-12-17 23:12:18 [INFO] HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
23:12:19 - LiteLLM:INFO: utils.py:2765 - 
LiteLLM completion() model= gpt-4o; provider = openai
2024-12-17 23:12:19 [INFO] 
LiteLLM completion() model= gpt-4o; provider = openai


...



# Agent: Research Expert
## Thought: The search results are still not providing the necessary information on the causes of the 1929 Great Depression. It seems there might be an issue with the search tool or the available documents. I will attempt one last search with a different approach, focusing on the stock market crash and economic downturn.
## Using tool: vector_search
## Tool Input: 
"{\"query\": \"1929 stock market crash and economic downturn causes\"}"
## Tool Output: 
Document 1:
----------------------------------------
Why did the world enter a global depression in 1929 ?

Document 2:
----------------------------------------
When was `` the Great Depression '' ?

Document 3:
----------------------------------------
What are bear and bull markets ?

Document 4:
----------------------------------------
When did the Dow first reach ?

Document 5:
----------------------------------------
What historical event happened in Dogtown in 1899 ?

Document 6:
--------------------------

2024-12-17 23:12:44 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
23:12:44 - LiteLLM:INFO: utils.py:894 - Wrapper: Completed Call, calling success_handler
2024-12-17 23:12:44 [INFO] Wrapper: Completed Call, calling success_handler
23:12:44 - LiteLLM:INFO: utils.py:2765 - 
LiteLLM completion() model= gpt-4o; provider = openai
2024-12-17 23:12:44 [INFO] 
LiteLLM completion() model= gpt-4o; provider = openai




# Agent: Research Expert
## Final Answer: 
The 1929 Great Depression was caused by a combination of factors, including:

1. Stock Market Crash of 1929: The stock market crash in October 1929 is often cited as the immediate trigger of the Great Depression. The crash led to a loss of confidence in the financial system and a reduction in consumer spending and investment.

2. Bank Failures: Many banks failed during the early 1930s, leading to a contraction in the money supply. This was partly due to bank runs, where panicked depositors withdrew their funds, causing even solvent banks to collapse.

3. Reduction in Consumer Spending: As people lost their jobs and savings, consumer spending decreased significantly, leading to a decline in production and further job losses.

4. International Trade Issues: The imposition of tariffs, such as the Smoot-Hawley Tariff, led to a decline in international trade. Other countries retaliated with their tariffs, exacerbating the global economic downturn

2024-12-17 23:13:03 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
23:13:03 - LiteLLM:INFO: utils.py:894 - Wrapper: Completed Call, calling success_handler
2024-12-17 23:13:03 [INFO] Wrapper: Completed Call, calling success_handler




# Agent: Technical Writer
## Final Answer: 
**The Causes of the 1929 Great Depression: A Comprehensive Analysis**

**Introduction**

The Great Depression, which began in 1929 and lasted until the late 1930s, was the most severe economic downturn in modern history. It had profound effects on both the United States and the global economy, leading to widespread unemployment, poverty, and political upheaval. Understanding the causes of the Great Depression is crucial for comprehending the economic and social dynamics of the early 20th century. This analysis explores the key factors that contributed to the onset of the Great Depression, including the stock market crash, bank failures, reduction in consumer spending, international trade issues, and monetary policy missteps.

**Main Body**

**a. Stock Market Crash of 1929**

The stock market crash of October 1929 is often regarded as the immediate trigger of the Great Depression. In the years leading up to the crash, the stock market experi